In [2]:
import pandas as pd

In [3]:
hop_teaming = pd.read_csv('Hop_Teaming_Data/DocGraph_Hop_Teaming_2018.csv', 
                          nrows = 100)

In [5]:
hop_teaming.head(10)

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508062167,1730166109,350,370,53.922,72.612
1,1508065640,1730166109,25,25,49.800,55.006
2,1508052093,1730166109,16,16,109.500,70.593
3,1508172545,1730166109,14,14,103.357,75.483
4,1508285131,1730166109,20,21,89.952,89.880
5,1508252933,1730166109,16,16,78.750,103.169
6,1508076001,1730166109,46,48,72.625,83.263
7,1508085911,1730166125,58,67,23.925,43.923
8,1508093394,1730166125,15,15,37.400,79.602
9,1508132986,1730166125,11,11,97.364,62.152


In [7]:
#Create chunk
chunks = pd.read_csv('Hop_Teaming_Data/DocGraph_Hop_Teaming_2018.csv', chunksize=10000)

In [8]:
next(chunks)

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508062167,1730166109,350,370,53.922,72.612
1,1508065640,1730166109,25,25,49.800,55.006
2,1508052093,1730166109,16,16,109.500,70.593
3,1508172545,1730166109,14,14,103.357,75.483
4,1508285131,1730166109,20,21,89.952,89.880
...,...,...,...,...,...,...
9995,1508026980,1730477589,36,46,64.239,77.845
9996,1497997050,1730477589,38,43,40.395,60.289
9997,1497977268,1730477761,14,14,37.857,72.533
9998,1508024217,1730477811,12,12,59.000,46.043


**We want to eliminate "accidental" referrals, so filter the hop teaming data so that the transaction_count is at least 50 and the average_day_wait is less than 50.**

In [9]:
chunks = pd.read_csv('Hop_Teaming_Data/DocGraph_Hop_Teaming_2018.csv', chunksize=10000)
accident_filtered = []
for chunk in chunks:
    chunk = chunk[(chunk["transaction_count"] >= 50) & (chunk["average_day_wait"] < 50)]
    accident_filtered.append(chunk)

In [10]:
len(accident_filtered)

21792

In [12]:
hop = pd.concat(accident_filtered, ignore_index = True)

In [13]:
hop.shape

(34176938, 6)

**First, build a profile of providers referring patients to the major hospitals in Nashville. Are certain specialties more likely to refer to a particular hospital over the others?**

SQLITE

In [11]:
import sqlite3
from tqdm.notebook import tqdm